In [ ]:
# Divide and conquer matafakas, cache results for faster processing. Run time under 0.1 sec
# Solved using tree model where each leaf node is a individual problem. 
from functools import cache # love this

with open('input.txt') as file:
    data = file.read()

data = data.split('\n')
data = ['A' + row for row in data]
mapping = {'0':0,'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,'A':10}
numbers = [int(row[1:-1]) for row in data]
inputs = [[mapping[x] for x in row] for row in data]

# Pretty self explatonory. keypad 10 = A, directional_keypad 4 = A
keypad = {7: (0, 0), 8: (0, 1),9: (0, 2),4: (1, 0),5: (1, 1),6: (1, 2),1: (2, 0),2: (2, 1),3: (2, 2),0: (3, 1),10: (3, 2)}
directional_keypad = {0: (0,1), 1: (1,0), 2:(1,1), 3:(1,2), 4:(0,2)}


def render_seq(seq): # For debugging
    mapping = {0:'^',1:'<',2:'v',3:'>',4:'A'}
    s = ''
    for x in seq:
        s += mapping[x]
    return s

@cache
def key_presses(next_pos,prev_pos, keypad):
    # Function returns needed key presses

    # These probably could have been written better but couldnt care less.
    blocked = {0:(3,0),1:(0,0)}
    up = max(prev_pos[0]-next_pos[0],0)
    down = max(next_pos[0]-prev_pos[0],0)
    left = max(prev_pos[1]-next_pos[1],0)
    right = max(next_pos[1]-prev_pos[1],0)

    seq = [4] + left * [1] + down * [2] + up * [0] + right * [3] + [4]

    # This was the tricky part to figure out. Basicly every sequence consist of two instructions. 
    # If we would go to the blocked cell reverse order of these instructions.

    # Top 1 most cursed if statements
    if (prev_pos[0], prev_pos[1] - left) == blocked[keypad] \
    or (prev_pos[0]+down,prev_pos[1]) == blocked[keypad] \
    or (prev_pos[0]-up,prev_pos[1]) == blocked[keypad]:
        seq.reverse()
    
    return seq
        
@cache
def solve_tree(seq,depth=0,max_depth=1,verbal=False):
    # Neat recursion function. Very mindful, very demure
    score = 0
    for ii in range(1,len(seq)):
        # Get key presses
        if depth == 0:
            seq_ii = key_presses(keypad[seq[ii]],keypad[seq[ii-1]],keypad=0)
        else:
            seq_ii = key_presses(directional_keypad[seq[ii]],directional_keypad[seq[ii-1]],keypad=1)

        if verbal: # For debuggind
            print(depth * 5 * ' ',render_seq(seq_ii))

        if depth != max_depth:
            score += solve_tree(tuple(seq_ii),depth + 1,max_depth,verbal) # Solve leaf node
        else:
            score += len(seq_ii)-1
        
    return score

s = 0
for input_kp,number in zip(inputs,numbers):
    s += solve_tree(tuple(input_kp),max_depth=25) * number # max_depth = 2 for silver 

s